# Vectores, operaciones vectoriales básicas y Universal functions



Como cientificos de datos en algunos casos requeriremos crear nuestras propias funciones y es recomendable que se apeguen a las ufuncs, es decir que se puedan aplicar independiente de la dimensión del objeto al que se aplican, que soporten broadcasting y muy importante **que su definición evite ciclos y use operaciones vectorizadas y otras ufuncs** .

In [33]:
import numpy as np

## Ejercicio 1
En data science y programación científica es común necesitar vectores con valores dentro de cierto intervalo, pero con sub-intervalos internos con diference espaciamiento ,por ejemplo:

[0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ]
       
El primer sub-intervalo incrementa de 0.1 en 0.1 , el segundo de 0.01 en 0.01 , y el tercero de 0.1 en 0.1 nuevamente.

Usando unicamente NumPy crea un vector de este tipo con el nombre xs .

**nota**: no es permitido usar ciclos, debe realizarse con operaciones "vectorizadas" de numpy

In [35]:
## tu codigo aqui (~ 4 lineas de codigo)
ar1 = np.linspace(0,0.5,6,endpoint=True,retstep=False)
ar2 = np.linspace(ar1[len(ar1)-1]+.01,0.6,10,endpoint=True,retstep=False)
ar3 = np.linspace(ar2[len(ar2)-1]+.1,1,4,endpoint=True,retstep=False)

np.hstack((ar1,ar2,ar3))

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ])

## Ejercicio 2
Suavizado de curvas a través de promedios de n-puntos(medias móviles):

<img src="https://www.luisllamas.es/wp-content/uploads/2017/03/arduino-filtro-media-movil-ventana-5.png">

En el mundo la data comunmente posee "ruido" por lo cual muchas veces antes de hacer análisis o modelado se necesita "suavizar" las curvas para reducir este ruido, una técnica muy sencilla es  el suavizado por promedio de n-puntos(también llamado media móvil), esto significa que para cada punto Xn obtenemos una versión transformada(suavizada) que consiste en promediar n puntos cercanos a el, por ejemplo para n= 3 tenemos que:

$$XS_{n}  =  \frac{X_{n-1}+X_{n}+X_{n+1}}{3}$$

In [36]:
# las siguientes 3 lineas generan un conjunto de datos que se comportan segun una onda senoidal pero tienen
# ruido por lo cual vamos a suavizar usando promedio de 3 puntos
# en este ejercicio los generamos manualmente pero pensemos que son datos que pudieron ser generados con algún
# instrumento como un sensor, o bien estar almacenados en una base de datos
ruido = 0.1*np.random.randn(15) #el ruido comunmente se debe a aleatoriedad o captura no exacta de info.
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

#xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()

for n in range(1,len(x)-1):
    xs[n] = (x[n-1] + x[n] + x[n+1])/3
    
print(x)
print(xs)

[ 0.05343141  0.33022413  0.76603025  0.95806951  0.95943995  0.75751511
  0.55737752 -0.04117625 -0.3905892  -0.66824927 -0.99827186 -0.98552097
 -0.61978501 -0.43332861 -0.19813494]
[ 0.05343141  0.38322859  0.68477463  0.89451324  0.89167486  0.75811086
  0.42457213  0.04187069 -0.36667157 -0.68570345 -0.88401404 -0.86785928
 -0.67954486 -0.41708285 -0.19813494]


En este caso usamos un ciclo  para ejemplificar pero ya hemos mencionado que debemos evitarlos siempre que sea posible y usar operaciones "vectorizadas" ya que es mucho mas rápido al utilizar  código pre-compilado de C, en este ejercicio debemos remplazar el calculo de xs para hacerlo de manera "vectorizada" y eliminar el ciclo for. 

**tip**: usar slicing

In [45]:
# Poniendo este codigo como texto porque el array ya fue creado en
# el codigo anterior y si no, mi comprobacion no da lo mismo.
# ruido = 0.1*np.random.randn(15) 
# x = np.linspace(0,2*np.pi,15) + ruido
# x = np.sin(x)

# xs es "x suavizado", inicialmente es una copia de x
# xs =  x.copy()

## tu codigo aqui (~ 1 linea):
num = 3

# La funcion Movave() funciona para arrays de cualquier largo, pero solo para "n"s impares

def movave(array,nume):
    begend = int((nume - 1)/2)
    final_array = np.hstack((array[0:begend],((array[begend:(len(array)-begend)] + array[0:(len(array)-begend*2)] + array[(begend*2):(len(array))]) / 3),array[(len(array)-begend):len(array)]))
    return final_array

print(movave(x,num))

print(xs)

movave(x,num) == xs

[ 0.05343141  0.38322859  0.68477463  0.89451324  0.89167486  0.75811086
  0.42457213  0.04187069 -0.36667157 -0.68570345 -0.88401404 -0.86785928
 -0.67954486 -0.41708285 -0.19813494]
[ 0.05343141  0.38322859  0.68477463  0.89451324  0.89167486  0.75811086
  0.42457213  0.04187069 -0.36667157 -0.68570345 -0.88401404 -0.86785928
 -0.67954486 -0.41708285 -0.19813494]


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

## Ejercicio 3

NumPy posee logaritmos naturales , base 2 y base 10 , crear una *ufunc* que calcule el logaritmo en cualquier base n dada, la funcion debe recibir el tensor al cual calcular el logaritmo elemento por elemento y la base del logaritmo a calcular, por ejemplo:

`def logaritmo_n(x,n):`

In [38]:
## tu codigo aqui:
def logaritmo_n(x,n):
    return np.log(x) / np.log(n)

logaritmo_n([10,28,36],5)


array([1.43067656, 2.07041507, 2.22656551])

## Ejercicio 4
Para calcular la información de una distribución de probabilidad necesitamos calcular el producto entre la información de cada posible $x$ y su probabilidad, cada uno de estos elementos nos indica cuanta incerteza aporta cada $x$ a la entropía de la distribución. 

Para completar este ejercicio hay que crear una función que calcule lo descrito anteriormente, debe recibir un vector representando a la dist. de probabilidad y devoler otro vector con el termino de entropía para cada $x$.

Por ejemplo:

`calcular_entropia([0.25,0.75])`

Debe resultar en :

$[0.5    ,    0.31127812]$

**Nota**: la entropía es la suma sobre estos valores, en este caso no calculamos la entropía,unicamente los termimos de su sumatoria. La entropía es:

$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

In [39]:
## tu codigo aqui (~ 2 lineas de codigo)
vect = ([0.25,0.75])

def calcular_entropia(vectp):
    vecto = vectp * np.log2(vectp) * -1
    return vecto

print(calcular_entropia(vect))


[0.5        0.31127812]


## Ejercicio 5

A partir de la información en un data warehouse se ha generado un modelo de datos, el valor de los clientes en el tiempo se ha determinado un modelo predictivo  no lineal que estima cuanta ganancia ha generado cierto cliente en 5 años en base a lo que ha gastado en su primer mes como cliente. 

El modelo tiene la forma:

$$g(x) = (x^{3} + 2x^{2} + e^{0.0001x} -\sqrt{2x})/50$$

Crear una función que calcule la ganancia para valores de gasto de nuevos clientes:

In [40]:
def g(x):
    ganancia = (np.power(x,3) + 2.0*(np.power(x,2.0))+np.exp(0.0001*x) - np.sqrt(2.0*x))/50
    
    return ganancia
    
gastos = np.array([100,50,25])
ganancia = g(gastos)

print("Ganancias",ganancia)

Ganancias [20399.73735829  2599.82010025   337.37862871]


Se ha determinado también que hay costos asociados a la comunicación y relación con los clientes(llamadas,mensajes electrónicos,etc) y que estos también tienen alta correlación con el monto que estos gastan en su primer més siguiendo el modelo:
$$c(x) = (x^{2} + x + log (0.0001x)-\sqrt{0.54x})/100$$

Cree una función para calcular este modelo sobre valores de de gasto de nuevos clientes , y luego cree una función  **ganancia_neta(x,g,c)** donde:
* x = vector conteniendo los valores de gasto del primer mes para nuevos clientes.
* g = función que estima la ganancia en función de x
* c = función que estima el costo en función de x

La función realiza el calculo simple **g(x) - c(x)** , ya que es posible que los modelos cambien en el tiempo es común recibir como parámetro el modelo o modelos hijos de un modelo mas grande , en este caso eso se logra recibiendo como parámetro las funciones de ganancia y costo.

In [41]:
## tu codigo aqui :
#Funcion para Costos

def c(x):
    costos = (np.power(x,2) + x + np.log10(0.0001*x) - np.sqrt(0.54*x))/100
    
    return costos

costo = c(gastos)

print("Costos: ",costo)


#Funcion para Ganancia Neta

def gan_neta(x,g,c):
    ganan_neta = g(x) - c(x)
    return ganan_neta

ganancia_neta = gan_neta(gastos,g,c)

print("Ganancia Neta: ",ganancia_neta) == print(ganancia-costo)


Costos:  [100.90651531  25.42502818   6.43723705]
Ganancia Neta:  [20298.83084298  2574.39507207   330.94139165]
[20298.83084298  2574.39507207   330.94139165]


True

# Funciones miscelaneas(agregación y estadistica descriptiva básica) 

Ya hemos trabajado algunas funciones de NumPy pero han sido funciones "elementwise" cuyo vector resultante tiene el mismo tamaño que el vector de entrada, existen muchas otras funciones que utilizamos en casos en los que necesitamos agregar,resumir cierto vector,incluyendo en este tipo de funciones algunas de estadistica descriptiva básica.

## Ejercicio 6
Usando argmax crear una función `mode(x)` para calcular la moda de un vector $x$, esta función debe devolver una tupla de 2 elementos de la forma:

(valor,conteo)

**tip** investigar y auxiliarse de otras funciones de numpy y np.argmax, no olvidar trabajar todo vectorizado, sin ciclos.

In [42]:
def mode(x):
    ## tu codigo aqui (~ 3 lineas de codigo):
    info_data = np.unique(x,return_index=True,return_counts=True)
    index_of_mode = np.argmax(info_data[2])
    return np.array(info_data)[0,index_of_mode],np.array(info_data)[2,index_of_mode]
print(mode(x))

(-0.9982718642604294, 1.0)


## Ejercicio 7
Crear una función para calcular la entropía  de una distribución de probabilidad representada como un vector.
$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

**Nota**: a diferencia del ejercicio anterior donde solo calculamos los términos de la entropía, en este caso  si calculamos el valor completo de la entropía, que como mencionamos es una medida de incerteza en una distribución de probabilidad.

**recordatorio**: no usar ciclos solo operaciones vectorizadas.

In [43]:
# tu codigo aqui (~ 3 lineas de codigo)
def calc_entropia(w):
    vecto = w * np.log2(w) * -1
    return np.sum(vecto)

print(calc_entropia(vect))

0.8112781244591328
